In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.base_env import ActionTuple
from collections import deque
from SAC_beta.replaybuffer import ReplayBuffers
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [226]:
class Actor(nn.Module):

    def __init__(self, action_dim, state_dim):
        super(Actor, self).__init__()

        self.action_dim = action_dim
        self.std_bound = [1e-2, 1.0]

        self.h1 = nn.Linear(state_dim, 128)
        self.h2 = nn.Linear(128, 64)
        self.h3 = nn.Linear(64, 32)
        self.h4 = nn.Linear(32, 16)
        self.mu = nn.Linear(16, action_dim)
        self.std = nn.Linear(16, action_dim)

    def forward(self, state):
        x = nn.functional.relu(self.h1(state))
        x = nn.functional.relu(self.h2(x))
        x = nn.functional.relu(self.h3(x))
        x = nn.functional.relu(self.h4(x))
        mu = torch.tanh(self.mu(x))
        std = nn.functional.softplus(self.std(x))

        std = torch.clamp(std, self.std_bound[0], self.std_bound[1])

        return mu, std

    def sample_normal(self, mu, std):
        normal_prob = Normal(mu, std)
        action = normal_prob.sample()

        # limit the action value
        log_prob = normal_prob.log_prob(action)
        log_prob = torch.sum(log_prob, dim=1, keepdim=True)

        return action, log_prob


In [227]:
class Critic(nn.Module):

    def __init__(self, action_dim, state_dim):
        super(Critic, self).__init__()

        self.x1 = nn.Linear(state_dim, 128)
        self.a1 = nn.Linear(action_dim, 128)
        # this layer is responsible for taking mixed state_action len.
        self.h = nn.Sequential(
            # 256 because it will be connected to two input tensors
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )


    def forward(self, state_action):
        state = state_action[0]
        action = state_action[1]
        x = nn.functional.relu(self.x1(state))
        a = nn.functional.relu(self.a1(action))
        h = torch.cat((x, a), dim=-1)
        q = self.h(h)
        return q


In [228]:
class SACagent(object):
    def __init__(self, N_STATES, N_ACTIONS):
        # Hyperparameters
        self.GAMMA = 0.99
        self.BATCH_SIZE = 1000
        self.BUFFER_SIZE = 10000
        self.ACTOR_LEARNING_RATE = 0.0001
        self.CRITIC_LEARNING_RATE = 0.001
        self.TAU = 0.001
        self.ALPHA = 0.5

        # Observation space and Action space
        self.state_dim = N_STATES
        self.action_dim = N_ACTIONS

        # Check if CUDA is available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Build Actor, Q1, Q2 and their target networks
        self.actor = Actor(action_dim=self.action_dim, state_dim=self.state_dim)


        self.critic_1 = Critic(action_dim=self.action_dim, state_dim=self.state_dim)
        self.target_critic_1 = Critic(action_dim=self.action_dim, state_dim=self.state_dim)

        self.critic_2 = Critic(action_dim=self.action_dim, state_dim=self.state_dim)
        self.target_critic_2 = Critic(action_dim=self.action_dim, state_dim=self.state_dim)

        # self.target_critic_1.load_state_dict(self.critic_1.state_dict())
        # self.target_critic_2.load_state_dict(self.critic_2.state_dict())

        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=self.ACTOR_LEARNING_RATE)
        self.critic_1_optimizer = optim.Adam(self.critic_1.parameters(), lr=self.CRITIC_LEARNING_RATE)
        self.critic_2_optimizer = optim.Adam(self.critic_2.parameters(), lr=self.CRITIC_LEARNING_RATE)

        # Clear out the buffer
        self.buffer = ReplayBuffers(self.BUFFER_SIZE)

        # For plotting purposes, data is stored.
        self.policy_loss = []
        self.reward_list = []

    def get_action(self, state):
        with torch.no_grad():
            state = torch.tensor([state], dtype=torch.float32).to(self.device)
            mu, std = self.actor(state)
            normal = torch.distributions.Normal(mu, std)
            action = normal.sample()
            return action.cpu().numpy()

    def update_target_network(self, TAU):
        phi_1 = self.critic_1.state_dict()
        phi_2 = self.critic_2.state_dict()
        target_phi_1 = self.target_critic_1.state_dict()
        target_phi_2 = self.target_critic_2.state_dict()
        for name in phi_1:
            target_phi_1[name] = TAU * phi_1[name] + (1 - TAU) * target_phi_1[name]
            target_phi_2[name] = TAU * phi_2[name] + (1 - TAU) * target_phi_2[name]
        self.target_critic_1.load_state_dict(target_phi_1)
        self.target_critic_2.load_state_dict(target_phi_2)


    # train Q1, Q2
    def critic_learn(self, states, actions, y_i):
        states = torch.tensor(states, dtype=torch.float32).to(self.device)
        actions = torch.tensor(actions, dtype=torch.float32).to(self.device)
        q_targets = y_i.to(self.device)
        q_1 = self.critic_1([states, actions])
        # where q_1 is the predicted value, q_targets is the true val.
        loss_1 = F.mse_loss(q_1, q_targets)

        # sets the gradients of all parameters of the optimizer to zero. This is necessary to prevent the gradients from accumulating from multiple backpropagation passes.
        self.critic_1_optimizer.zero_grad()
        #computes the gradients of the loss with respect to all the learnable parameters in the critic network
        loss_1.backward()
        #updates the parameters of the critic network based on the computed gradients.
        self.critic_1_optimizer.step()

        q_2 = self.critic_2([states, actions])
        loss_2 = F.mse_loss(q_2, q_targets)

        self.critic_2_optimizer.zero_grad()
        loss_2.backward()
        self.critic_2_optimizer.step()

    def actor_learn(self, states):
        states = torch.tensor(states, dtype=torch.float32).to(self.device)
        mu, std = self.actor(states)
        actions, log_pdfs = self.actor.sample_normal(mu, std)
        log_pdfs = log_pdfs.squeeze(1)
        soft_q_1 = self.critic_1([states, actions])
        soft_q_2 = self.critic_2([states, actions])
        soft_q = torch.min(soft_q_1, soft_q_2)

        loss = torch.mean(self.ALPHA * log_pdfs - soft_q)

        self.actor_optimizer.zero_grad()
        loss.backward()
        self.actor_optimizer.step()

        return float(loss)

    def q_target(self, rewards, q_values, dones):
        y_k = np.asarray(q_values)
        for i in range(q_values.shape[0]):
            if dones[i]:
                y_k[i] = rewards[i]
            else:
                y_k[i] = rewards[i] + self.GAMMA * q_values[i]
        return torch.tensor(y_k, dtype=torch.float32)

    def load_weights(self, path):
        self.actor.load_state_dict(torch.load(path + 'Drone_actor_2q.pth'))
        self.critic_1.load_state_dict(torch.load(path + 'Drone_critic_12q.pth'))
        self.critic_2.load_state_dict(torch.load(path + 'Drone_critic_22q.pth'))

    def train(self, max_episode_num, env, behavior_name):
        self.param_counter = 0
        cnt = 0
        # reset target network param.
        self.update_target_network(1.0)

        for ep in range(int(max_episode_num)):
            frame, episode_reward = 0, 0
            # reset the enviornment
            env.reset()
            decision_steps, terminal_steps = env.get_steps(behavior_name)
            episode_done = False
            # setting up the initial state as an array
            x = decision_steps.obs[0][0] # Ray Perception 3D
            y = decision_steps.obs[1][0] # Agent's velocity x,z
            state = np.concatenate((x, y), 0)

            while not episode_done:

                action = self.get_action(state)
                # wrap the action with ActionTuple before sending it to UE.
                action = ActionTuple(np.array(action, dtype = np.float32))
                env.set_actions(behavior_name, action)
                # move the agent along with the action.
                env.step()
                action = action._continuous # converting ActionTuple to array
                next_decision_steps, next_terminal_steps = env.get_steps(behavior_name)

                # if the agent is still on, collect data and add it to buffer.
                if next_decision_steps:
                    # get the reward.
                    train_reward = next_decision_steps.reward[0]
                    x = next_decision_steps.obs[0][0]
                    y = next_decision_steps.obs[1][0]
                    next_state = np.concatenate((x, y), 0)
                    episode_reward += next_decision_steps.reward[0]
                    # store the data to the buffer
                    self.buffer.add_data(state, action, train_reward, next_state, False)
                    episode_done = False

                # if the agent is off, collect data and add True for done.
                if next_terminal_steps:
                    # get the reward.
                    train_reward = next_terminal_steps.reward[0]
                    x = next_terminal_steps.obs[0][0]
                    y = next_terminal_steps.obs[1][0]
                    next_state = np.concatenate((x, y), 0)
                    episode_reward += next_terminal_steps.reward[0]
                    # store the data to the buffer
                    self.buffer.add_data(state, action, train_reward, next_state, True)
                    episode_done = True

                # if buffer has enough data start training.
                if self.buffer.buffer_count() > self.BATCH_SIZE:

                    states, actions, rewards, next_states, dones = self.buffer.sample_batch(self.BATCH_SIZE)

                    # Calculate the Q target value
                    with torch.no_grad():
                        next_mu, next_std = self.actor(torch.tensor(next_states, dtype=torch.float32))
                        next_actions, next_log_pdf = self.actor.sample_normal(next_mu, next_std)

                        # convert np to tensor
                        tensor_next_states = torch.tensor(next_states, dtype=torch.float32)
                        tensor_next_actions = torch.tensor(next_actions, dtype=torch.float32)

                        # move to CUDA
                        tensor_next_states = tensor_next_states.to(self.device)
                        tensor_next_actions = tensor_next_actions.to(self.device)

                        target_qs_1 = self.target_critic_1([tensor_next_states, tensor_next_actions])
                        target_qs_2 = self.target_critic_2([tensor_next_states, tensor_next_actions])
                        target_qs = torch.min(target_qs_1, target_qs_2)

                        target_qi = target_qs - self.ALPHA * next_log_pdf
                        y_i = self.q_target(rewards, target_qi.numpy(), dones)


                    self.critic_learn(states, actions, y_i)

                    # update Actor and return policy loss
                    policy_loss = self.actor_learn(states)

                    # store the performance of the algorithm.
                    if cnt % 500 == 0:
                        self.reward_list.append(train_reward)
                        self.policy_loss.append(policy_loss)

                    self.update_target_network(self.TAU)

                state = next_state
                frame += 1
                cnt += 1

            # Episode output
            print('Episode: ', ep+1, 'Frame: ', frame, 'u Reward: ', episode_reward)

            # every 250th run will store another params
            if max_episode_num % 250:
                torch.save(self.actor.state_dict(), "./saved_weights/250th/actor_"+self.param_counter+"_2q.pth")
                torch.save(self.critic_1.state_dict(), "./saved_weights/250th/critic_"+self.param_counter+"_12q.pth")
                torch.save(self.critic_2.state_dict(), "./saved_weights/250th/critic_"+self.param_counter+"_22q.pth")
                self.param_counter += 1
            # Save weights for each run
            torch.save(self.actor.state_dict(), "./saved_weights/Drone_actor_2q.pth")
            torch.save(self.critic_1.state_dict(), "./saved_weights/Drone_critic_12q.pth")
            torch.save(self.critic_2.state_dict(), "./saved_weights/Drone_critic_22q.pth")

    def plot_result(self):
        fig=plt.figure(figsize=(18, 6))
        fig.add_subplot(1, 3, 1)  # 1 row, 3 columns
        plt.plot(self.reward_list)

        fig.add_subplot(1, 3, 3)
        plt.plot(self.policy_loss)

        plt.show()



#### Run Unity Enviornment

In [229]:
N_ACTIONS = 3

In [230]:
env = UnityEnvironment(file_name= "./LinuxBuilds/Linux_Drone_v0.0.1/Linux_Drone_with.x86_64", base_port=5004)

In [231]:
env.reset()
behavior_names = list(env.behavior_specs.keys())
behavior_name = behavior_names[0]
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [232]:
# initial states
N_STATES = len(decision_steps.obs[0][0]) + len(decision_steps.obs[1][0])

In [234]:
agent = SACagent(N_STATES, N_ACTIONS)
# usually 30K is enough.
agent.train(30000, env, behavior_name)

Episode:  1 Frame:  145 u Reward:  -0.013421893941945044
Episode:  2 Frame:  130 u Reward:  -0.014001699594350962
Episode:  3 Frame:  120 u Reward:  -0.017528661092122397
Episode:  4 Frame:  135 u Reward:  -0.007852568449797453
Episode:  5 Frame:  180 u Reward:  -0.009620157877604167
Episode:  6 Frame:  89 u Reward:  -0.024540933330407303


/var/folders/bx/11k7rc0x5xl576ckk4jkh6ch0000gn/T/ipykernel_14231/2943630160.py:172: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_next_actions = torch.tensor(next_actions, dtype=torch.float32)
/var/folders/bx/11k7rc0x5xl576ckk4jkh6ch0000gn/T/ipykernel_14231/2943630160.py:184: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_i, dtype=torch.float32))


Episode:  7 Frame:  301 u Reward:  -0.019364886109615086


KeyboardInterrupt: 

In [ ]:
agent.plot_result()

In [235]:
env.close()